In [115]:
import pandas as pd
import numpy as np
import sqlite3

In [116]:
#Load data
google_df = pd.read_csv('/Users/marianaarrieta/Documents/Data_Projects/london_restaurants/raw_data/data_google_maps.csv')
opentable_df = pd.read_csv('/Users/marianaarrieta/Documents/Data_Projects/london_restaurants/raw_data/opentable_data.csv')


In [117]:
google_df.head()

,names,summary,cusine,price,reviews_count,rating,address,url
0,Restaurant Story,Chef Tom Sellers' novel approach to British cl...,Fine dining restaurant,££££,418 reviews,4.6,"199 Tooley St, London SE1 2JX",https://www.google.com/maps/place/Restaurant+S...
1,Galvin La Chapelle,NaN,French restaurant,££££,"1,901 reviews",4.6,NaN,https://www.google.com/maps/place/Galvin+La+Ch...
2,City Social,NaN,Modern European restaurant,££££,"1,187 reviews",4.5,NaN,https://www.google.com/maps/place/City+Social/...
3,Brawn,Mediterranean small plates menu with an 'all t...,Mediterranean restaurant,££,615 reviews,4.7,"49 Columbia Rd, London E2 7RG",https://www.google.com/maps/place/Brawn/@51.52...
4,Il Bordello,Old-school Italian restaurant with an authenti...,Italian restaurant,££,"1,170 reviews",4.5,"metropolitan wharf, 70 Wapping Wall, London E1...",https://www.google.com/maps/place/Il+Bordello/...


In [118]:
opentable_df.head()

,names,cuisine,price,reviews,rating,bookings,address
0,Seasons Mayfair,Fish,4,41.0,4.3,8.0,Mayfair
1,PAPA L's KITCHEN,African,4,257.0,4.6,6.0,St. James's
2,Coqbull Soho,Rotisserie Chicken,2,175.0,4.1,47.0,Soho
3,Ham Yard Bar and Restaurant,Modern European,2,3696.0,4.6,92.0,Soho
4,El Norte,Spanish,4,296.0,4.5,45.0,Mayfair


### Filter Google Data

In [119]:
#Remove duplicates, if any 
google_df.drop_duplicates(inplace = True)

#Remove rows where cusine = "Not a restaurant"
google_df = google_df[google_df.cusine != 'Not a restaurant']

#Remove rows where there are no ratings 
google_df = google_df[google_df.rating != 'Not Available']


### Filter Open Table Data

In [120]:
#Remove duplicates, if any 
opentable_df.drop_duplicates(inplace = True)

#Remove booking column 
opentable_df.drop(columns=['bookings'], inplace = True)

## Understand the Differences in Data

In [121]:
google_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342 entries, 0 to 377
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   names          342 non-null    object
 1   summary        204 non-null    object
 2   cusine         341 non-null    object
 3   price          342 non-null    object
 4   reviews_count  342 non-null    object
 5   rating         342 non-null    object
 6   address        263 non-null    object
 7   url            342 non-null    object
dtypes: object(8)
memory usage: 24.0+ KB


In [122]:
opentable_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 499
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   names    400 non-null    object 
 1   cuisine  400 non-null    object 
 2   price    400 non-null    int64  
 3   reviews  358 non-null    float64
 4   rating   350 non-null    float64
 5   address  400 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 21.9+ KB


In [123]:
#Different values in price
print(google_df.price.unique())
print(opentable_df.price.unique())

#Price ranges from 1 to 4, but we need to have the same format across tables
price_dict = {'£':1, '££':2, '£££':3, '££££':4, 'Not Available': np.nan}
#Map values
google_df['price'] = google_df['price'].map(price_dict)


['££££' '££' 'Not Available' '£££' '£']
[4 2 3]


In [126]:
#Clean reviews  and ratings before converting to float
google_df['reviews_count'] = google_df['reviews_count'].replace(',','', regex=True)
google_df['reviews_count'] = google_df['reviews_count'].replace('[a-zA-Z]','', regex=True)
google_df['reviews_count'] = google_df['reviews_count'].replace('',np.nan, regex=True)
google_df['reviews_count'] = google_df['reviews_count'].replace(""" ''""", np.nan, regex=True)
google_df['reviews_count'] = google_df['reviews_count'].str.strip()

google_df['rating'] = google_df['rating'].replace('Not Available', np.nan, regex=True)

In [127]:
google_df['reviews_count'] = google_df['reviews_count'].astype(float)

google_df['rating'] = google_df['rating'].astype(float)

In [128]:
#Make sure caps are the same in restaurant titles to be able to merge later
google_df['names'] = google_df['names'].str.lower()
google_df['names'] = google_df['names'].str.title()
opentable_df['names'] = opentable_df['names'].str.lower()
opentable_df['names'] = opentable_df['names'].str.title()

In [129]:
#Make column names suitable for merging 
google_df.rename({'cusine': 'Google Cuisine', 'price': 'Google Price', 'rating': 'Google Rating', 'reviews_count': "Google Review Count"}, axis=1, inplace=True)

In [130]:
opentable_df.rename({'cuisine': 'OpenTable Cuisine', 'price': 'OpenTable Price', 'rating': 'OpenTable Rating', 'reviews': "OpenTable Review Count"}, axis=1, inplace=True)

In [132]:
restaurants_df = pd.merge(google_df, opentable_df, how="inner", on=["names"])

In [135]:
len(google_df)

342

In [ ]:
pd.set_option('display.max_rows', 1000)
opentable_df.names.unique()

array(['Seasons Mayfair', "Papa L'S Kitchen", 'Coqbull Soho',
       'Ham Yard Bar And Restaurant', 'El Norte', 'Park Row',
       "Bill'S Restaurant & Bar - Soho", "Whitcomb'S", 'Temper Soho',
       'Maoz Soho', 'The Ivy Soho Brasserie', 'Happy',
       'Gaucho Piccadilly', 'Pierre Victoire', 'Forty Dean Street',
       'Tamarind Kitchen', 'Fishworks - Swallow Street', 'Robata',
       'Prix Fixe Brasserie', 'Sarap Bistro', 'Heddon Street Kitchen',
       'Farzi Cafe', 'Lobos Meat & Tapas - Soho', "Rowley'S Restaurant",
       'Estiatorio Milos – London', "Vasco & Piero'S Pavilion Restaurant",
       'Green Bar At Hotel Cafe Royal', 'Orient London',
       'Zima Russian Restaurant', 'Kanishka', 'Ino Gastrobar',
       'Pix Carnaby', 'Refuel Restaurant At The Soho Hotel',
       "Ruby'S Soho", 'The Rainforest Cafe London', 'Aqua Kyoto',
       'Sola Soho', 'Macellaio Rc Soho', 'Suvlaki Soho',
       'Red Dog Saloon Soho', 'The Good Egg Soho', 'Patara - Soho',
       'Obicà Mozzarella 